# Orchestrator Test

Smoke test for `ModelsOrchestrator` — data loading, MLP training, LightGBM training, and combined prediction.

In [ ]:
import sys
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score

from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from models_classes.models_orchestrator import ModelsOrchestrator
from models_classes.mlp_disease_neural_net import device

print(f'PyTorch: {torch.__version__}')
print(f'Using device: {device}')

## 1. Load data

In [ ]:
orchestrator = ModelsOrchestrator(type_disease='chikungunya')

x_train_cat, x_test_cat, x_train_num, x_test_num, y_train, y_test, embedding_sizes = orchestrator.prepare_data()
numerical_columns = orchestrator.df.drop(columns=list(orchestrator.categorical_columns) + ['final_classification']).columns

print(f'Train: {len(y_train)} | Test: {len(y_test)}')
print(f'Categorical features: {len(orchestrator.categorical_columns)}')
print(f'Numerical features:   {len(numerical_columns)}')

## 2. Train MLP

In [ ]:
mlp_model = orchestrator.train_mlp(embedding_sizes=embedding_sizes,save_path='C:\\Users\\angej\\Documents\\2_Programação\\health_index_project\\models_saved\\best_orchestrator_mlp.pth')

### 2.1 MLP evaluation

In [ ]:
display(mlp_model.evaluate(orchestrator.test_loader, orchestrator.y_test))

## 3. Train LightGBM

In [ ]:
lgbm_model = orchestrator.train_lgbm(x_train_cat, x_train_num, y_train)
lgbm_model.evaluate(x_test_cat, x_test_num, y_test, orchestrator.categorical_columns, numerical_columns)

## 3. Train XGB

In [ ]:
xgb_model = orchestrator.train_xgb(x_train_cat, x_train_num, y_train)
xgb_model.evaluate(x_test_cat, x_test_num, y_test, orchestrator.categorical_columns, numerical_columns)

## 4. Combined prediction (MLP + LightGBM average)

In [ ]:
confirmation_df = orchestrator.evaluate_combined(mlp_model, lgbm_model, xgb_model, x_test_cat, x_test_num)
display(confirmation_df['all_agree'].value_counts())
display(confirmation_df.head(10))